In [1]:
# basic libs
import numpy as np

# cleaning data
import re
import os
import nltk
nltk.download("stopwords")
nltk.download('punkt')

# save vocabulary in files
import pickle

# tokenization
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Model
from tensorflow.keras.layers import LSTM,Embedding,Input,Dense,SpatialDropout1D,Activation
from tensorflow.keras.models import Model,load_model


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\engah\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\engah\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
model=load_model("Model_english_to_frensh.h5")

In [3]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 encoder_input_layer (InputLaye  [(None, None)]      0           []                               
 r)                                                                                               
                                                                                                  
 encoder_embedding_layer (Embed  (None, None, 300)   4171200     ['encoder_input_layer[0][0]']    
 ding)                                                                                            
                                                                                                  
 decoder_input_layer (InputLaye  [(None, None)]      0           []                               
 r)                                                                                           

In [4]:
model.get_layer("decoder_lstm_layer")

In [5]:
word_2_idx_target=pickle.load(open("vocabulary/word_2_idx_target.txt","rb"))
word_2_idx_target["end"]

1

In [6]:
len(word_2_idx_target)

26942

In [7]:
word_2_idx_input=pickle.load(open("vocabulary/word_2_idx_input.txt","rb"))
len(word_2_idx_input)

13904

In [8]:
num_encoder_tokens=13902
max_encoder_sequence_len=47
num_decoder_tokens=26560
max_encoder_sequence_len=57

In [9]:
def make_references():
  encoder_input=Input(shape=(None,),name="encoder_input_layer")
  encoder_embedding=model.get_layer("encoder_embedding_layer")(encoder_input)
  encoder_lstm=model.get_layer("encoder_lstm_1_layer")(encoder_embedding)
  encoder_lstm2=model.get_layer("encoder_lstm_2_layer")(encoder_lstm)
  _,state_h,state_c=encoder_lstm2
  encoder_states=[state_h,state_c]
  encoder_reference_model=Model(encoder_input,encoder_states)


  decoder_state_h=Input(shape=(256,))
  decoder_state_c=Input(shape=(256,))
  decoder_input_states=[decoder_state_h,decoder_state_c]

  decoder_input=Input(shape=(None,),name="decoder_input_layer")
  decoder_embedding=model.get_layer("decoder_embedding_layer")(decoder_input)
  decoder_lstm=model.get_layer("decoder_lstm_layer")

  decoder_outputs,state_h,state_c=decoder_lstm(decoder_embedding,initial_state=encoder_states)
  decoder_dense=model.get_layer("deocer_final_layer")
  
  decoder_outputs,state_h,state_c=decoder_lstm(decoder_embedding,initial_state=decoder_input_states)

  decoder_state=[state_h,state_c]
  decoder_outputs=decoder_dense(decoder_outputs)
  decoder_reference_model=Model([decoder_input]+decoder_input_states,[decoder_outputs]+decoder_state)

  return encoder_reference_model,decoder_reference_model

In [10]:
arr=np.array([[59, 13,  2,  0,  0]], dtype='int32')
arr

array([[59, 13,  2,  0,  0]])

In [11]:
# clean english column
def clean_english(text):
  text=text.lower() # lower case

  # remove any characters not a-z and ?!,'
  text=re.sub(u"[^a-z!?',]"," ",text)

  # word tokenization
  text=nltk.word_tokenize(text)

  # join text
  text=" ".join([i.strip() for i in text])

  return text


def prepare_text(text):
  text=clean_english(text)

  res=[word_2_idx_input[i] for i in text.split(" ")]
  pad=pad_sequences([res],maxlen=5,padding="post")
  return pad
prepare_text("How are you")

array([[43, 23,  2,  0,  0]])

In [12]:
def translate_eng_fr(text):
    enc_model,dec_model=make_references()

    states_value=enc_model(prepare_text(text))

    empty_target_seq=np.zeros((1,1))
    empty_target_seq[0,0]=word_2_idx_target["start"]

    stop_condition=False
    decoded_translaition=""

    while not stop_condition:
      dec_output,h,c=dec_model.predict([empty_target_seq]+states_value)
      sampled_word_index=np.argmax(dec_output[0,-1,:])
      sampled_word=None

      for word,index in word_2_idx_target.items():
        if sampled_word_index == index:
          decoded_translaition+=' {}'.format(word)
          sampled_word=word

        if sampled_word == "end" or len(decoded_translaition.split(" ")) >= 12:
          stop_condition=True

      empty_target_seq=np.zeros((1,1))
      empty_target_seq[0,0]=sampled_word_index
      states_value=[h,c]
    decoded_translaition=" ".join(decoded_translaition.split()[:-1])
    print(decoded_translaition)

In [13]:
translate_eng_fr(input("Enter text :- "))

Enter text :- hello
1/1 [==============================] - 0s 26ms/step
qu'as tu du bon train


In [14]:
translate_eng_fr("wait")

1/1 [==============================] - 0s 27ms/step
j'attends


In [15]:
translate_eng_fr("stop it")

1/1 [==============================] - 0s 26ms/step
j'attends arrête ordinateur prochaine


In [16]:
translate_eng_fr("try do this")

1/1 [==============================] - 0s 27ms/step
j'apprécie d'essayer


In [17]:
translate_eng_fr("try for this")

1/1 [==============================] - 0s 28ms/step
j'apprécie de se faire


In [18]:
translate_eng_fr("help")

1/1 [==============================] - 0s 27ms/step
qu'as décédé


In [19]:
translate_eng_fr("i just")

1/1 [==============================] - 0s 26ms/step
je ne suis pas du


In [20]:
translate_eng_fr("My Car")

1/1 [==============================] - 0s 26ms/step
j ai ma voiture


In [22]:
translate_eng_fr("i love you !")

1/1 [==============================] - 0s 26ms/step
qu'est ce que vous !


In [23]:
translate_eng_fr("he can")

1/1 [==============================] - 0s 27ms/step
il a raison


In [24]:
translate_eng_fr("i am happy")

1/1 [==============================] - 0s 27ms/step
je vous ?


In [25]:
translate_eng_fr("in my home")

1/1 [==============================] - 0s 26ms/step
qu'avez dans ma maison


In [26]:
translate_eng_fr("my home")

1/1 [==============================] - 0s 26ms/step
qu'avez chez moi


In [27]:
translate_eng_fr("i have money")

1/1 [==============================] - 0s 27ms/step
j'ai eu de l'argent


In [28]:
translate_eng_fr("i have a car")

1/1 [==============================] - 0s 27ms/step
j ai une voiture


In [29]:
translate_eng_fr("i lost my car")

1/1 [==============================] - 0s 29ms/step
j'ai perdu ma partie


In [33]:
translate_eng_fr("i have Car")

1/1 [==============================] - 0s 28ms/step
j ai une voiture


In [34]:
translate_eng_fr("i play football")

1/1 [==============================] - 0s 69ms/step
je joue au football


In [36]:
translate_eng_fr("i play")

1/1 [==============================] - 0s 27ms/step
je de jouer


In [38]:
translate_eng_fr("she play football")

1/1 [==============================] - 0s 27ms/step
il joue au football


In [39]:
translate_eng_fr("i won a prize")

1/1 [==============================] - 0s 28ms/step
j'ai eu un nouvel appartement


In [40]:
translate_eng_fr("I'm done with you.")

1/1 [==============================] - 0s 27ms/step
je vous suis sorti de h


In [41]:
translate_eng_fr("I'm eating a pear.")

1/1 [==============================] - 0s 46ms/step
je sais de l'intérieur


In [43]:
translate_eng_fr("happy birthday")

1/1 [==============================] - 0s 27ms/step
j'imagine avec vous


In [44]:
translate_eng_fr("Can I call you again?")

1/1 [==============================] - 0s 27ms/step
malheureusement tu de nouveau ?


In [45]:
translate_eng_fr("Stop! You're hurting him!")

1/1 [==============================] - 0s 27ms/step
j'apprécie tout le temps


In [47]:
translate_eng_fr("he work player")

1/1 [==============================] - 0s 41ms/step
il y a eu un travail


In [48]:
translate_eng_fr("hello")

1/1 [==============================] - 0s 44ms/step
qu'as tu du bon train


In [50]:
translate_eng_fr("how are you")

1/1 [==============================] - 0s 27ms/step
qu'as tu de la place


In [51]:
translate_eng_fr("i see.")

1/1 [==============================] - 0s 47ms/step
je vous ai vu


In [52]:
translate_eng_fr("I can't wait for you.")

1/1 [==============================] - 0s 27ms/step
j'attends exactement
